In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import accuracy_score, confusion_matrix
np.random.seed(512)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.layers import LeakyReLU
from keras import initializers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from keras.utils import to_categorical
import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,StratifiedKFold
from keras.models import load_model

Using TensorFlow backend.


In [2]:
# define network parameters
max_features = 128
maxlen = 512

In [3]:
#read csv
dict_label = {'Usable directly numeric':0, 'Usable with extraction':1, 'Usable with Extration': 1, 'Usable with extraction ':1, 'Usable directly categorical':2, 'Unusable':3, 'Context_specific':4, 'Usable directly categorical ':2}
data = pd.read_csv('data_for_ML_num.csv')
data['y_act'] = [dict_label[i] for i in data['y_act']]
y = data.loc[:,['y_act']]
data_LSTM = pd.concat([data['Attribute_name'], data['sample_1'], data['sample_2'], data['sample_3'], data['sample_4'], data['sample_5']], axis =1)

# X_train, X_test,y_train,y_test = train_test_split(data,y, test_size=0.2)

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2,5,10,11,12,13,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data = data.rename(columns={'Num of nans': 'Num_of_nans', 'num of dist_val': 'num_of_dist_val'})

data['Num_of_nans'] = [data['Num_of_nans'][i]*100/data['Total_val'][i] for i in data.index]
data['num_of_dist_val'] = [data['num_of_dist_val'][i]*100/data['Total_val'][i] for i in data.index]

data1 = data[['Num_of_nans', 'max_val', 'mean', 'min_val', 'num_of_dist_val','std_dev','castability','extractability', 'len_val']]
data1 = data1.fillna(0)
print('hghj')
data1 = data1.rename(columns={'mean': 'scaled_mean', 'min_val': 'scaled_min_val', 'max_val': 'scaled_max_val','std_dev': 'scaled_std_dev'})
data1.loc[data1['scaled_min_val'] > 10000, 'scaled_min_val'] = 10000
data1.loc[data1['scaled_min_val'] < -10000, 'scaled_min_val'] = -10000
data1.loc[data1['scaled_max_val'] > 10000, 'scaled_max_val'] = 10000
data1.loc[data1['scaled_max_val'] < -10000, 'scaled_max_val'] = -10000
data1.loc[data1['scaled_mean'] > 10000, 'scaled_mean'] = 10000
data1.loc[data1['scaled_mean'] < -10000, 'scaled_mean'] = -10000
data1.loc[data1['scaled_std_dev'] > 10000, 'scaled_std_dev'] = 10000
data1.loc[data1['scaled_std_dev'] < -10000, 'scaled_std_dev'] = -10000
column_names_to_normalize = ['scaled_max_val', 'scaled_mean', 'scaled_min_val','scaled_std_dev']
x = data1[column_names_to_normalize].values
x = np.nan_to_num(x)
x_scaled = StandardScaler().fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data1.index)
data1[column_names_to_normalize] = df_temp

data1.Num_of_nans = data1.Num_of_nans.astype(float)
data1.num_of_dist_val = data1.num_of_dist_val.astype(float)
data1.castability = data1.castability.astype(float)
data1.extractability = data1.extractability.astype(float)
y.y_act = y.y_act.astype(float)


print('hghj')
X_train, X_test,y_train,y_test = train_test_split(data1,y, test_size=0.2,random_state=100)

key_name = data['Attribute_name']
atr_train,atr_test = train_test_split(key_name, test_size=0.2,random_state=100)
# print(atr_train)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
atr_train.reset_index(inplace=True,drop=True)
atr_test.reset_index(inplace=True,drop=True)

list_sentences_train = atr_train.values
list_sentences_test = atr_test.values


structured_data_train = X_train 
structured_data_test = X_test

structured_input_train = []
for i in range(len(structured_data_train)):
#     print(structured_data_train.loc[i])
#     print(list(structured_data_train.loc[i]))
    structured_input_train.append(list(structured_data_train.loc[i]))
structured_input_train = np.array(structured_input_train).reshape(len(structured_data_train),len(structured_data_train.keys()))    

structured_input_test = []
for i in range(len(structured_data_train)):
    structured_input_test.append(list(structured_data_train.loc[i]))
structured_input_test = np.array(structured_input_test).reshape(len(structured_data_train),len(structured_data_train.keys()))    


hghj
hghj


In [5]:
key_name = data['sample_1']
samp1_train,samp1_test = train_test_split(key_name, test_size=0.2,random_state=100)
samp1_train.reset_index(inplace=True,drop=True)
samp1_test.reset_index(inplace=True,drop=True)


key_name = data['sample_2']
samp2_train,samp2_test = train_test_split(key_name, test_size=0.2,random_state=100)
samp2_train.reset_index(inplace=True,drop=True)
samp2_test.reset_index(inplace=True,drop=True)

In [6]:

list_sentences_train = atr_train.values
list_sentences_test = atr_test.values

# X_train.sample_1 = X_train.sample_1.astype(str)
# X_test.sample_1 = X_test.sample_1.astype(str)

list_sentences_train1 = samp1_train.values
list_sentences_test1 = samp1_test.values

print(list_sentences_train1)

# X_train.sample_2 = X_train.sample_2.astype(str)
# X_test.sample_2 = X_test.sample_2.astype(str)

list_sentences_train2 = samp2_train.values
list_sentences_test2 = samp2_test.values

print(list_sentences_train2)

for i in range(len(list_sentences_train)):
    list_sentences_train[i] = str(list_sentences_train[i])
    list_sentences_train1[i] = str(list_sentences_train1[i])
    list_sentences_train2[i] = str(list_sentences_train2[i])
    
for i in range(len(list_sentences_test)):
    list_sentences_test[i] = str(list_sentences_test[i])
    list_sentences_test1[i] = str(list_sentences_test1[i])
    list_sentences_test2[i] = str(list_sentences_test2[i])    


['#NULL!' '0' '268' ... 'Hate' '-0.101' '2']
['0' '78' '110' ... nan '0.171' '5']


In [7]:
tokenizer = keras_text.Tokenizer(char_level = True)
tokenizer.fit_on_texts(list(list_sentences_train))
# train data
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=maxlen)
# test data
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = keras_seq.pad_sequences(list_tokenized_test, maxlen=maxlen)


tokenizer1 = keras_text.Tokenizer(char_level = True)
tokenizer1.fit_on_texts(list(list_sentences_train1))
# train data
list_tokenized_train1 = tokenizer.texts_to_sequences(list_sentences_train1)
X_t1 = keras_seq.pad_sequences(list_tokenized_train1, maxlen=maxlen)
# test data
list_tokenized_test1 = tokenizer.texts_to_sequences(list_sentences_test1)
X_te1 = keras_seq.pad_sequences(list_tokenized_test1, maxlen=maxlen)


tokenizer2 = keras_text.Tokenizer(char_level = True)
tokenizer2.fit_on_texts(list(list_sentences_train1))
# train data
list_tokenized_train2 = tokenizer.texts_to_sequences(list_sentences_train2)
X_t2 = keras_seq.pad_sequences(list_tokenized_train2, maxlen=maxlen)
# test data
list_tokenized_test2 = tokenizer.texts_to_sequences(list_sentences_test2)
X_te2 = keras_seq.pad_sequences(list_tokenized_test2, maxlen=maxlen)

# print(tokenizer)

In [8]:
def build_model(numfilters,embed_size):
#     embed_size = 512
    inp = Input(shape=(None, ))
    x = Embedding(input_dim = len(tokenizer.word_counts)+1, output_dim = embed_size)(inp)
    prefilt_x = Dropout(0.25)(x)
    out_conv = []

    x = prefilt_x
    for i in range(2):
        x = Conv1D(numfilters*2**(i), kernel_size = 3, activation = 'relu')(x)
    out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    out_conv += [GlobalMaxPool1D()(x)]
    xy = concatenate(out_conv, axis = -1)  
    print(xy.shape)
    #########################################################
    inp1 = Input(shape=(None, ))
    x = Embedding(input_dim = len(tokenizer1.word_counts)+1, output_dim = embed_size)(inp1)
    prefilt_x = Dropout(0.25)(x)
    out_conv = []

    x = prefilt_x
    for i in range(2):
        x = Conv1D(numfilters*2**(i), kernel_size = 3, activation = 'relu')(x)
    out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    out_conv += [GlobalMaxPool1D()(x)]
    x1 = concatenate(out_conv, axis = -1)
    print(x1.shape)
#     x1 = Flatten()(out_conv)
    #########################################################
#     inp2 = Input(shape=(None, ))
#     x = Embedding(input_dim = len(tokenizer2.word_counts)+1, output_dim = embed_size)(inp1)
# #     prefilt_x = Dropout(0.25)(x)
#     out_conv = []
#     # dilation rate lets us use ngrams and skip grams to process 
#     for dilation_rate in dilation_rates:
# #         x = prefilt_x
#         for i in range(2):
#             if dilation_rate>0:
#                 x = Conv1D(16*2**(i), 
#                            kernel_size = 3, 
#                            dilation_rate = dilation_rate,
#                           activation = 'relu',
#                           name = 'ngram_{}_cnn2_{}'.format(dilation_rate, i)
#                           )(x)
#             else:
#                 x = Conv1D(16*2**(i), 
#                            kernel_size = 1,
#                           activation = 'relu',
#                           name = 'word_fcl2_{}'.format(i))(x)
# #         out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
#         out_conv += [GlobalMaxPool1D()(x)]
#     x2 = concatenate(out_conv, axis = -1)
#     print(x2.shape)
#     x1 = Flatten()(out_conv)
#     #########################################################
    Str_input = Input(shape=(9,))
    layersfin = keras.layers.concatenate([xy,x1,Str_input])
    print(layersfin.shape)
    x = Dense(1000, activation='relu')(layersfin)
    x = Dropout(0.1)(x)
    x = Dense(1000, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(5, activation='softmax')(x)
    model = Model(inputs=[inp,inp1,Str_input], outputs=[x])
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

# model = build_model()
# model.summary()

In [9]:
batch_size = 32 # large enough that some other labels come in
epochs = 50

file_path="best_weights.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=20)

callbacks_list = [checkpoint, early] #early
# history = model.fit([X_t,X_t1,structured_data_train], to_categorical(y_train),
#                     validation_data=([X_te,X_te1,structured_data_test], to_categorical(y_test)),
#                     batch_size=batch_size, epochs=epochs, shuffle = True, callbacks=callbacks_list)

# # model = build_model()
# # model.summary()


In [10]:
# print(X_t)
print(X_t.shape)
# print(y_train[1851:])

y_train = y_train.values
structured_data_train = structured_data_train.values

(7400, 512)


In [13]:
# # n_filters_grid = [16,32,64,128]
# # embed_size = [32,64,128,512]

# batch_size = 32 # large enough that some other labels come in
# epochs = 50

# k = 5
# kf = KFold(n_splits=k)
# n_filters_grid = [32]
# embed_size = [512]

# avgsc,avgsc_train = 0,0
# for train_index, test_index in kf.split(X_t):
#     print('\n')
#     X_train_cur, X_test_cur = X_t[train_index], X_t[test_index]
#     y_train_cur, y_test_cur = y_train[train_index], y_train[test_index]
#     structured_data_train_cur, structured_data_test_cur = structured_data_train[train_index],structured_data_train[test_index]
    
#     X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=100)
#     structured_data_train_train,structured_data_val = train_test_split(structured_data_train_cur, test_size=0.25,random_state=100)
    
#     print(y_train_train.shape)
#     print(y_val.shape)
#     print(structured_data_train_train.shape)
#     print(structured_data_val.shape)
    
#     bestPerformingModel = build_model(10,5)
#     bestscore = 0
#     for ne in n_filters_grid:
#         for md in embed_size:
# #             clf = RandomForestClassifier(n_estimators=ne,max_depth=md)
#             clf = build_model(ne,md)
#             history = clf.fit([X_train_train,structured_data_train_train], to_categorical(y_train_train),
#                     validation_data=([X_val,structured_data_val], to_categorical(y_val)),
#                     batch_size=batch_size, epochs=epochs, shuffle = True, callbacks=callbacks_list)
        
#             clf.load_weights('best_weights.h5')
#             sc = clf.evaluate([X_val,structured_data_val],to_categorical(y_val))
#             print(sc)
#             if bestscore < sc:
#                 bestscore = sc
#                 bestPerformingModel = clf
# #                 print(bestPerformingModel)

# #     bscr_train = bestPerformingModel.score(X_train_cur, y_train_cur)
# #     bscr = bestPerformingModel.score(X_test_cur, y_test_cur)
    
#     bestPerformingModel.load_weights('best_weights.h5')
#     loss, bscr_train = bestPerformingModel.evaluate([X_train_train,structured_data_train_train],to_categorical(y_train_train))
#     print(loss, bscr_train)
#     loss, bscr = bestPerformingModel.evaluate([X_test_cur,structured_data_test_cur],to_categorical(y_test_cur))
#     print(loss, bscr)
    
#     avgsc = avgsc + bscr
#     avgsc_train = avgsc_train + bscr_train
#     print('The training accuracy is:')
#     print(bscr_train)
#     print('The test accuracy is:')    
#     print(bscr)
#     print('\n')
    
# print(avgsc_train/k)
# print(avgsc/k)

In [13]:
# from keras.models import load_model
# # model = create_model()

# clf = build_model(ne,md)
# clf.load_weights('best_weights.h5')

# loss, acc = clf.evaluate([X_train_train,structured_data_train_train],to_categorical(y_train_train))
# print(loss, acc)
# loss, acc = clf.evaluate([X_test_cur,structured_data_test_cur],to_categorical(y_test_cur))
# print(loss, acc)

In [ ]:
n_filters_grid = [16,32,64,128]
embed_size = [32,64,128,512]

# n_filters_grid = [16]
# embed_size = [32,64]

batch_size = 32 # large enough that some other labels come in
epochs = 50

k = 5
kf = KFold(n_splits=k)
# n_filters_grid = [32]
# embed_size = [256]

avgsc_lst,avgsc_train_lst = [],[]
avgsc,avgsc_train = 0,0
bestne,bestmd = 0,0
i=0
for train_index, test_index in kf.split(X_t):
    print('\n')
    X_train_cur, X_test_cur = X_t[train_index], X_t[test_index]
    X_train_cur1, X_test_cur1 = X_t1[train_index], X_t1[test_index]    
    y_train_cur, y_test_cur = y_train[train_index], y_train[test_index]
    structured_data_train_cur, structured_data_test_cur = structured_data_train[train_index],structured_data_train[test_index]
    
    X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=100)
    X_train_train1, X_val1 = train_test_split(X_train_cur1, test_size=0.25,random_state=100)    
    structured_data_train_train,structured_data_val = train_test_split(structured_data_train_cur, test_size=0.25,random_state=100)
    
    print(y_train_train.shape)
    print(y_val.shape)
    print(structured_data_train_train.shape)
    print(structured_data_val.shape)
    
    bestPerformingModel = build_model(10,5)
    bestscore = 0
    for ne in n_filters_grid:
        for md in embed_size:
            file_path= 'best_weights'+str(ne)+str(md)+'.h5'
            checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
            early = EarlyStopping(monitor="val_acc", mode="max", patience=20)
            callbacks_list = [checkpoint, early] #early            
            
#             clf = RandomForestClassifier(n_estimators=ne,max_depth=md)
            clf = build_model(ne,md)
            history = clf.fit([X_train_train,X_train_train1,structured_data_train_train], to_categorical(y_train_train),
                    validation_data=([X_val,X_val1,structured_data_val], to_categorical(y_val)),
                    batch_size=batch_size, epochs=epochs, shuffle = True, callbacks=callbacks_list)
        
            clf.load_weights('best_weights'+str(ne)+str(md)+'.h5')
            sc = clf.evaluate([X_val,X_val1,structured_data_val],to_categorical(y_val))
            print(sc)
        
            if bestscore < sc:
                bestscore = sc
                bestPerformingModel = clf
                bestne,bestmd = ne,md
                print(bestPerformingModel)
    
#     bestPerformingModel.load_weights('best_weights.h5')
    bestPerformingModel.load_weights('best_weights'+str(bestne)+str(bestmd)+'.h5')
    loss, bscr_train = bestPerformingModel.evaluate([X_train_train,X_train_train1,structured_data_train_train],to_categorical(y_train_train))
    print(loss, bscr_train)
    loss, bscr = bestPerformingModel.evaluate([X_test_cur,X_test_cur1,structured_data_test_cur],to_categorical(y_test_cur))
    print(loss, bscr)
    
    avgsc_lst.append(bscr)
    avgsc_train_lst.append(bscr_train)      
    
    avgsc = avgsc + bscr
    avgsc_train = avgsc_train + bscr_train
    print('The training accuracy is:')
    print(bscr_train)
    print('The test accuracy is:')    
    print(bscr)
    print('\n')
    i=i+1


print(avgsc_train/k)
print(avgsc/k)



(5550, 1)
(1850, 1)
(5550, 9)
(1850, 9)
(?, 40)
(?, 40)
(?, 89)
(?, 64)
(?, 64)
(?, 137)
Train on 5550 samples, validate on 1850 samples
Epoch 1/50
5550/5550 [==============================] - 10s 2ms/step - loss: 2.0837 - acc: 0.5220 - val_loss: 1.3214 - val_acc: 0.6324

Epoch 00001: val_acc improved from -inf to 0.63243, saving model to best_weights1632.h5
Epoch 2/50
5550/5550 [==============================] - 3s 475us/step - loss: 1.2270 - acc: 0.6411 - val_loss: 0.8029 - val_acc: 0.7092

Epoch 00002: val_acc improved from 0.63243 to 0.70919, saving model to best_weights1632.h5
Epoch 3/50
5550/5550 [==============================] - 3s 463us/step - loss: 0.7656 - acc: 0.7133 - val_loss: 0.7026 - val_acc: 0.7454

Epoch 00003: val_acc improved from 0.70919 to 0.74541, saving model to best_weights1632.h5
Epoch 4/50
5550/5550 [==============================] - 3s 461us/step - loss: 0.6954 - acc: 0.7447 - val_loss: 0.6603 - val_acc: 0.7714

Epoch 00004: val_acc improved from 0.74541 t

In [2]:
print(avgsc_train_lst)
print(avgsc_lst)

print('The training accuracy is:')
print(avgsc_train/k)
print('The test accuracy is:')
print(avgsc/k)



(?, 40)
(?, 64)
(?, 49)
(?, 128)
(?, 64)
(?, 137)
Train on 5550 samples, validate on 1850 samples
Epoch 1/50
5550/5550 [==============================] - 10s 2ms/step - loss: 1.0227 - acc: 0.6114 - val_loss: 0.7779 - val_acc: 0.7443

Epoch 00001: val_acc improved from -inf to 0.74432, saving model to best_weights0.h5
Epoch 2/50
5550/5550 [==============================] - 3s 486us/step - loss: 0.7274 - acc: 0.7458 - val_loss: 0.6817 - val_acc: 0.7703

Epoch 00002: val_acc improved from 0.74432 to 0.77027, saving model to best_weights0.h5
Epoch 3/50
5550/5550 [==============================] - 3s 501us/step - loss: 0.5988 - acc: 0.7849 - val_loss: 0.5813 - val_acc: 0.7978

Epoch 00003: val_acc improved from 0.77027 to 0.79784, saving model to best_weights0.h5
Epoch 4/50
5550/5550 [==============================] - 3s 483us/step - loss: 0.5167 - acc: 0.8267 - val_loss: 0.5144 - val_acc: 0.8243

Epoch 00004: val_acc improved from 0.79784 to 0.82432, saving model to best_weights0.h5
Epoc